In [3]:
import pandas as pd
import json

In [4]:
# Load data fetched from SEC API using sec_edgar_fetcher.py
df_edgar = pd.read_csv("../Data/sec_data/pivoted_financial_data.csv")

In [ ]:
# Extract the missing values, will be used to help with feature selection
missing_edgar = df_edgar.isna().sum()
missing_edgar = missing_edgar.sort_values()
missing_edgar.head(30)

ticker                    0
date                      0
IncomeTax               639
DilutedShares          1304
WeightedShares         1457
Assets                 1596
Liabilities            1648
FinancingCashFlow      2087
InvestingCashFlow      2129
NetIncome              2173
OperatingCashFlow      2209
StockholdersEquity     3075
PPE                    4051
Goodwill               4861
CurrentAssets          5569
CurrentLiabilities     5580
ShareBasedComp         6111
OperatingIncome        6209
InterestExpense        6625
CapEx                  7074
Revenue                8357
SGA                   10033
Amortization          10968
COGS                  12718
GrossProfit           12889
LongTermDebt          13430
RnD                   16396
OperatingExpenses     16606
dtype: int64

In [6]:
df_edgar.columns

Index(['ticker', 'date', 'Amortization', 'Assets', 'COGS', 'CapEx',
       'CurrentAssets', 'CurrentLiabilities', 'DilutedShares',
       'FinancingCashFlow', 'Goodwill', 'GrossProfit', 'IncomeTax',
       'InterestExpense', 'InvestingCashFlow', 'Liabilities', 'LongTermDebt',
       'NetIncome', 'OperatingCashFlow', 'OperatingExpenses',
       'OperatingIncome', 'PPE', 'Revenue', 'RnD', 'SGA', 'ShareBasedComp',
       'StockholdersEquity', 'WeightedShares'],
      dtype='object')

In [7]:
# Transforming json cik file to csv for easier use
import json

json_data = pd.read_json("../Data/company_tickers.json").T
json_data.columns = ["cik", "ticker", "company_name"]
json_data.to_csv("../Data/company_tickers.csv", index=False)

In [ ]:
# Reading data on 345 filings (insider trades) from Q1 2010.
# This data contains exact tickers what allows me to get CIKs for companies that are missing in the edgar data.

df345 = pd.read_csv("../Data/345-filings/SUBMISSION.tsv", sep='\t')
df345 = df345[['ISSUERCIK', 'ISSUERNAME', 'ISSUERTRADINGSYMBOL']].drop_duplicates()
df345.columns = ["cik", "company_name", "ticker"]
df345

,cik,company_name,ticker
0,355627,CTI GROUP HOLDINGS INC,CTIG
1,1289592,"Tower Group, Inc.",TWGP
2,1022345,MARINER ENERGY INC,ME
3,1401688,"Vitacost.com, Inc.",VITC
4,857005,PARAMETRIC TECHNOLOGY CORP,PMTC
...,...,...,...
69275,311817,HMG COURTLAND PROPERTIES INC,hmg
69353,1174228,"NURX PHARMACEUTICALS, INC.",NUXP
69394,1289636,PROFIRE ENERGY INC,PFIE
69628,1271057,"CHINA-BIOTICS, INC",CHBT


In [14]:
missing_df = pd.read_csv("../Data/sec_data/missing_ciks_2010vol1.csv")

df_ciks = missing_df.merge(df345, on="ticker", how="left")

In [21]:
company_tickers = pd.read_csv("../Data/company_tickers.csv")
tickers_new = pd.concat([company_tickers, df_ciks], axis=0)

# Handle missing values before converting to int
tickers_new['cik'] = tickers_new['cik'].fillna(-1).astype(int)

tickers_new.to_csv("../Data/company_tickers_2010_full.csv", index=False)

In [26]:
#Only 4 CIKs are missing now for 2010
cik_minus_one_count = (tickers_new['cik'] == -1).sum()
cik_minus_one_count

np.int64(4)